In [3]:
from sqlalchemy import create_engine
import pandas as pd

In [4]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [5]:
print(engine.table_names())

['Bookings', 'Facilities', 'Members']


<ipython-input-5-7f622051bbf8>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(engine.table_names())


In [16]:
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Facilities")
    fac = pd.DataFrame(rs.fetchall())
    fac.columns = rs.keys()

In [128]:
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Bookings")
    bookings = pd.DataFrame(rs.fetchall())
    bookings.columns = rs.keys()

In [123]:
with engine.connect() as con:
    rs = con.execute("SELECT * FROM Members")
    members = pd.DataFrame(rs.fetchall())
    members.columns = rs.keys()

### Q10: Produce a list of facilities with a total revenue less than 1000. 
#### The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [108]:
q10query = """
WITH guest AS(
    SELECT facid, SUM(slots) AS gslots
    FROM Bookings
    WHERE memid=0
    GROUP BY facid),    
    member AS (
    SELECT facid, SUM(slots) AS mslots
    FROM Bookings
    WHERE memid!=0
    GROUP BY facid)
SELECT f.name, 
    ((gslots*f.guestcost) + (mslots*f.membercost)) AS revenue
FROM Facilities AS f
INNER JOIN guest ON f.facid = guest.facid
INNER JOIN member ON f.facid = member.facid
ORDER BY revenue DESC;
"""

In [122]:
with engine.connect() as con:
    rs = con.execute(q10query)
    q10 = pd.DataFrame(rs.fetchall())
    q10.columns = rs.keys()
q10

,name,revenue
0,Massage Room 1,50351.6
1,Massage Room 2,14454.6
2,Tennis Court 2,14310.0
3,Tennis Court 1,13860.0
4,Squash Court,13468.0
5,Badminton Court,1906.5
6,Pool Table,270.0
7,Snooker Table,240.0
8,Table Tennis,180.0


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [145]:
q11query = """
SELECT m1.surname||', '||m1.firstname AS members,
    m2.surname||', '||m2.firstname AS rec_by
FROM Members AS m1
LEFT JOIN Members AS m2 ON m1.recommendedby = m2.memid
ORDER BY members;
"""

In [146]:
with engine.connect() as con:
    rs = con.execute(q11query)
    q11 = pd.DataFrame(rs.fetchall())
    q11.columns = rs.keys()
q11

,members,rec_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Farrell, David",None
9,"Farrell, Jemima",None


### Q12: Find the facilities with their usage by member, but not guests

In [172]:
q12query = """
SELECT f.name, sub.mslots AS member_usage
FROM Facilities AS f
LEFT JOIN
    (SELECT facid, SUM(slots) AS mslots
    FROM Bookings
    WHERE memid!=0
    GROUP BY facid) AS sub
    ON f.facid = sub.facid
ORDER BY member_usage DESC
"""

In [173]:
with engine.connect() as con:
    rs = con.execute(temp)
    q12 = pd.DataFrame(rs.fetchall())
    q12.columns = rs.keys()
q12

,name,member_usage
0,Badminton Court,1086
1,Tennis Court 1,957
2,Massage Room 1,884
3,Tennis Court 2,882
4,Snooker Table,860
5,Pool Table,856
6,Table Tennis,794
7,Squash Court,418
8,Massage Room 2,54


### Q13: Find the facilities usage by month, but not guests

In [206]:
q13query = """
SELECT f.name AS facility,
    STRFTIME('%m', b.starttime) AS month,
    SUM(slots) AS member_usage
FROM Bookings AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
WHERE memid!=0
GROUP BY facility, month
ORDER BY member_usage DESC;
"""

In [207]:
with engine.connect() as con:
    rs = con.execute(q13query)
    q13 = pd.DataFrame(rs.fetchall())
    q13.columns = rs.keys()
q13

,facility,month,member_usage
0,Badminton Court,09,507
1,Pool Table,09,443
2,Tennis Court 1,09,417
3,Badminton Court,08,414
4,Tennis Court 2,09,414
5,Snooker Table,09,404
6,Massage Room 1,09,402
7,Table Tennis,09,400
8,Tennis Court 2,08,345
9,Tennis Court 1,08,339
